In [1]:
from OKX_ws import run_okx_monitor
from create_bd import TradeDatabase
from historical_trades import OkxFillsFetcher
import asyncio
import threading
import traceback
import time
from concurrent.futures import ThreadPoolExecutor
from config import INSTRUMENT_IDS, CONFIG_DICT_TEST
from log.logger import get_logger


trader = 'ZERS_OKX'
api_key = '62106854-dd50-4aae-a770-67b4130bfcfd'
api_secret = '9E8372C785E05E8C9700A35FAC7A3C34'
passphrase = 'Neo1234!'

hist = OkxFillsFetcher(api_key=api_key, api_secret=api_secret, passphrase=passphrase)
db = TradeDatabase(CONFIG_DICT_TEST, trader)

hist_trade = hist.fetch_days(days=10)
# db.sync_trades_from_list(hist_trade)

      
  


In [3]:
for i in hist_trade:
    if i['tradeId'] == '1851196458':
        print(i)
        break

{'fillSz': '2.68', 'fillPxVol': '', 'fillFwdPx': '', 'fee': '-0.597256224', 'tradeQuoteCcy': '', 'ordId': '2979532103901470720', 'clOrdId': '', 'posSide': 'net', 'fillMarkVol': '', 'tag': '', 'execType': 'T', 'fillIdxPx': '111476.6', 'side': 'buy', 'fillPx': '111428.4', 'fillPnl': '-28.3241714153064341', 'instType': 'SWAP', 'fillPxUsd': '', 'instId': 'BTC-USDT-SWAP', 'billId': '2979532103968579584', 'subType': '1', 'fillTime': '1761299376325', 'tradeId': '1851196458', 'fillMarkPx': '111430.4', 'feeCcy': 'USDT', 'ts': '1761299376326', 'ordType': 'market'}


In [3]:
db.sync_trades_from_list(hist_trade)

{'inserted': 52, 'skipped': 148, 'errors': 0}

In [2]:
db.aggregate_and_send_unsent()

1 - ручное управление, 2 - автомат, 0 - остановить.
1 - ручное управление, 2 - автомат, 0 - остановить.


{'records': 2, 'sent': 2, 'failed': 0}

In [ ]:
# 2025-10-24 03:31:49,344 [INFO] utilites: Отправка сделки в Unity: провайдер=9855 клиент=24463 инструмент=87854 сторона=buy объём=2.68 цена=111428.4 ордер=1851196458
# 2025-10-24 03:31:49,578 [INFO] utilites: Ответ от Unity: статус=200, текст=
# 2025-10-24 03:31:54,382 [INFO] utilites: Отправка сделки в Unity: провайдер=9855 клиент=24463 инструмент=87854 сторона=buy объём=40.0 цена=111428.4 ордер=1851196457
# 2025-10-24 03:31:54,569 [INFO] utilites: Ответ от Unity: статус=200, текст=

trader = 'ZERS_OKX'

In [5]:
import sqlite3

# Подключение к базе данных
conn = sqlite3.connect('trades.db')
cursor = conn.cursor()

# ПРАВИЛЬНЫЙ запрос UPDATE
update_query = "UPDATE trades_ZERS_OKX SET is_send = 0 WHERE tradeId = 1851196458 OR tradeId = 1851196457"
cursor.execute(update_query)

# Подтверждаем изменения
conn.commit()

# Проверяем сколько строк обновлено
print(f"Обновлено строк: {cursor.rowcount}")

# Закрываем соединение
conn.close()

Обновлено строк: 2


In [16]:
import pandas as pd

df_trades =  pd.read_excel('tradesgrid_from_31_12_2024_22_00_to_22_10_2025_21_59_generated_22.xlsx')
df_trades['Amount'] = df_trades['Amount'].apply(lambda x: abs(x))
df_trades.head()

,ID,Instrument,Amount,Quote amount,Price,Side,Commission,Trade type,Trade date,Value date,Transact time,Converted P/L,Absolute amount
0,18503940,[CFD]BTCUSDT_PF.TOD :: OKXE,9.66,10450.34256,108181.600000,Sell,0,DIRECT,22.10.2025,22.10.2025,22.10.2025 13:36:04.000 (UTC+5),-,0.0966
1,18503942,[CFD]BTCUSDT_PF.TOD :: OKXE,9.67,10458.97530,108159.000000,Sell,0,DIRECT,22.10.2025,22.10.2025,22.10.2025 13:02:34.000 (UTC+5),-,0.0967
2,18503944,[CFD]BTCUSDT_PF.TOD :: OKXE,19.25,-20959.41925,108880.100000,Buy,0,DIRECT,21.10.2025,21.10.2025,22.10.2025 03:41:05.000 (UTC+5),-136.62859 USDT,0.1925
3,18503946,[CFD]BTCUSDT_PF.TOD :: OKXE,38.51,-41937.15894,108899.400000,Buy,0,DIRECT,21.10.2025,21.10.2025,22.10.2025 03:40:31.000 (UTC+5),-0.59232 USDT,0.3851
4,18503948,[CFD]BTCUSDT_PF.TOD :: OKXE,48.34,-52429.21852,108459.285312,Buy,0,DIRECT,21.10.2025,21.10.2025,22.10.2025 03:32:01.000 (UTC+5),-,0.4834


In [17]:
import sqlite3
import pandas as pd

# Подключение к базе данных
conn = sqlite3.connect('trades.db')  # укажите путь к вашей БД

# Способ 1: Чтение всей таблицы
df_bd_trades = pd.read_sql('SELECT * FROM trades_ZERS_OKX', conn)
df_bd_trades['side'] = df_bd_trades['side'].str.upper()

conn.close()

In [18]:
df_bd_trades.head()

,id,ordId,tradeId,instId,side,avgPx,state,uTime,fillSz,is_send,created_at,ordType
0,1,2973580687005409280,1844932683,BTC-USDT-SWAP,SELL,108181.6,filled,1761122164133,9.66,1,2025-10-22 01:37:45,limit
1,2,2973518367499706368,1844890213,BTC-USDT-SWAP,SELL,108159.0,filled,1761120154818,3.37,1,2025-10-22 01:37:45,limit
2,3,2973518367499706368,1844890212,BTC-USDT-SWAP,SELL,108159.0,filled,1761120154818,6.30,1,2025-10-22 01:37:45,limit
3,4,2972387992371159040,1843758663,BTC-USDT-SWAP,BUY,108880.1,filled,1761086465210,19.25,1,2025-10-22 01:37:45,market
4,5,2972386853869281280,1843756157,BTC-USDT-SWAP,BUY,108899.4,filled,1761086431280,38.51,1,2025-10-22 01:37:45,market


In [24]:
new_bd_trade = df_bd_trades[['side', 'avgPx', 'fillSz']].copy()
new_bd_trade['sorce'] = 'bd'
new_bd_trade['id'] = new_bd_trade.index
new_bd_trade.head()

,side,avgPx,fillSz,sorce,id
0,SELL,108181.6,9.66,bd,0
1,SELL,108159.0,3.37,bd,1
2,SELL,108159.0,6.30,bd,2
3,BUY,108880.1,19.25,bd,3
4,BUY,108899.4,38.51,bd,4


In [28]:
df_trades = df_trades[['Side', 'Price', 'Amount']].copy()
df_trades['sorce'] = 'Unity'
df_trades['id'] = df_trades.index
df_trades['Side'] = df_trades['Side'].str.upper()
df_trades.columns = new_bd_trade.columns
df_trades

,side,avgPx,fillSz,sorce,id
0,SELL,108181.600000,9.66,Unity,0
1,SELL,108159.000000,9.67,Unity,1
2,BUY,108880.100000,19.25,Unity,2
3,BUY,108899.400000,38.51,Unity,3
4,BUY,108459.285312,48.34,Unity,4
5,BUY,108666.000000,28.81,Unity,5
6,BUY,109750.749207,18.92,Unity,6
7,BUY,109597.600000,9.48,Unity,7
8,BUY,109601.400000,4.74,Unity,8
9,BUY,109586.600000,4.74,Unity,9


In [29]:
df_combined = pd.concat([new_bd_trade, df_trades], ignore_index=True)
df_combined

,side,avgPx,fillSz,sorce,id
0,SELL,108181.600000,9.66,bd,0
1,SELL,108159.000000,3.37,bd,1
2,SELL,108159.000000,6.30,bd,2
3,BUY,108880.100000,19.25,bd,3
4,BUY,108899.400000,38.51,bd,4
...,...,...,...,...,...
208,SELL,108069.500000,4.84,Unity,44
209,SELL,107650.000000,4.86,Unity,45
210,SELL,107989.200000,1.93,Unity,46
211,SELL,108206.236788,1.93,Unity,47


In [30]:
# Удаляем дубликаты по всем столбцам кроме sorce и id
df_no_duplicates = df_combined.drop_duplicates(
    subset=[col for col in df_combined.columns if col not in ['sorce', 'id']],
    keep='first'  # сохраняет первую встретившуюся запись
)

print(f"Было: {len(df_combined)} строк")
print(f"Стало: {len(df_no_duplicates)} строк")

Было: 213 строк
Стало: 180 строк


In [31]:
180 - 213

-33

In [32]:
df_no_duplicates

,side,avgPx,fillSz,sorce,id
0,SELL,108181.600000,9.66,bd,0
1,SELL,108159.000000,3.37,bd,1
2,SELL,108159.000000,6.30,bd,2
3,BUY,108880.100000,19.25,bd,3
4,BUY,108899.400000,38.51,bd,4
...,...,...,...,...,...
203,BUY,107035.845010,19.64,Unity,39
205,SELL,107426.400000,4.88,Unity,41
206,SELL,107746.455045,14.57,Unity,42
211,SELL,108206.236788,1.93,Unity,47
